In [ ]:
import sys
import os
from os.path import join
import time
from datetime import datetime
import importlib
import numpy as np
import pandas as pd
import h5py
import imageio
from scipy import ndimage
from scipy import interpolate
import skimage
from tqdm import tqdm
from tqdm import trange
from matplotlib import pyplot as plt
from matplotlib import colors
import plotly.graph_objs as go
from ipywidgets import interact
import proplot as pplt

sys.path.append('../..')
from tools import energyVS06 as energy
from tools import image_processing as ip
from tools import plotting as mplt
from tools import utils
from tools.utils import project

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

In [ ]:
folder = '.'

In [ ]:
shape = tuple(np.loadtxt(join(folder, 'a5d_new_shape.txt')).astype(int))
shape

In [ ]:
coords = utils.load_stacked_arrays('coords.npz')
for c in coords:
    print(c.shape)

In [ ]:
for c in coords:
    print(c)

In [ ]:
f = np.memmap(join(folder, 'a5d_new.mmp'), shape=shape, dtype='float', mode='r')

In [ ]:
f.shape

In [ ]:
f = np.clip(f, 0.0, None)

In [ ]:
for norm in [None, 'log']:
    axes = mplt.corner(
        f,
        coords=coords,
        fig_kws=dict(figwidth=1.5*4),
        cmap='viridis',
        diag_kind='None',
        prof='edges',
        prof_kws=dict(alpha=0.5, lw=0.9),
        labels=["x [mm]", "xp [mrad]", "y [mm]", "yp [mrad]", "w [MeV]"],
        norm=norm,
        frac_thresh=1e-5,
    )
    plt.savefig(f'_output/corner_int_norm{norm}.png')
    plt.show()